In [135]:
from whoosh.fields import Schema, TEXT, ID
from whoosh import index
import os, os.path
from whoosh import index
from whoosh import qparser
from whoosh.qparser import QueryParser
import pandas as pd
import numpy as np

In [136]:
schema = Schema(content_id=ID(stored=True), lesson=ID(stored=True), topic=TEXT(stored=True), content=TEXT(stored = True))

# create empty index directory

if not os.path.exists("index_dir"):
    os.mkdir("index_dir")

In [137]:
# Use the schema to initialize a Whoosh index in the above directory

ix = index.create_in("index_dir", schema)
writer = ix.writer()

In [138]:
# Read topic content file, then process it
df = pd.read_csv("cs510.csv")
i = 0
new_list = []

for _, row in df.iterrows():
    all_sentence = str(row["TEXT"]).split('.')
    for sentence in all_sentence:
        content_id = i
        i += 1
        lesson = row["Lesson"]
        topic = row["Topic"]
        content = sentence
        if content != "":
            while content[0] == ' ':
                content = content[1:]
            temp = [content_id, lesson, topic, content]
            new_list.append(temp)


In [139]:
for row in new_list:
    content_id = row[0]
    lesson = row[1]
    topic = row[2]
    content = row[3]
    writer.add_document(content_id=str(content_id), lesson=str(lesson), topic=str(topic), content=str(content))
writer.commit()

In [181]:
result_list = []

def index_search(dirname, search_fields, search_query):
    ix = index.open_dir(dirname)
    schema = ix.schema
    
    og = qparser.OrGroup.factory(0.9)
    mp = qparser.MultifieldParser(search_fields, schema, group = og)

    
    q = mp.parse(search_query)
    
    
    with ix.searcher() as s:
        results = s.search(q, terms=True, limit = 5)
#         print("Search Results: ")
        if results.is_empty():
            result_list.append("")
        else:
            result_list.append(results[0]['content'])
        return results

In [178]:
results = index_search("index_dir", ['topic', 'content'], "Since we avoid the too frequent words (e.g., 'the') for the initial idea, why not avoid them also in mixture LM examples by ignoring the less valuable words and smooth the frequency of other words likewise?")

In [182]:
questions = pd.read_csv("510questions.csv")
all_result = []
for _, row in questions.iterrows():
    index_search("index_dir", ['topic', 'content'], row["questions"])
    
print(result_list)

['There are several major types of language models, including statistical language models, rule-based language models, and neural language models', 'An N-gram language model is a statistical language model that uses a sequence of N words (an N-gram) to predict the next word in a text or speech', 'A statistical language model is a mathematical model that estimates the probability of a sequence of words', 'The model is trained using a set of training data, and the resulting model can be used to predict the probability of a sequence of hidden states', 'Cross entropy is a measure of the difference between two probability distributions', 'Mutual information is a measure of the amount of information that two variables share', 'Cross entropy is a measure of the difference between two probability distributions', 'Cross entropy is a measure of the difference between two probability distributions', 'One is that the probability of a word occurring is only dependent on the words that come before i

In [183]:
questions['answer'] = result_list

In [184]:
questions.to_csv('questions_with_result.csv')